## Introduction to data files





Today we are going to jump right into loading data from files into numpy arrays. Once we have the arrays, then we will then start learning how to use them.





## Loading data from a text file





Usually, data will be in a file, and we need to read it in. There are *many* data formats to get familiar with, and *many* ways to read them in. Let's start by looking at data from a Raman spectrometer.

In this directory there is a file called 01-raman.txt. The file name suggests it is Raman data, but only because of its name. The extension *suggests* this is text data, but does not tell us the format.

*You cannot in general rely on the extension to tell you what format a file is in. That is an advertisement, and only a convention that is used. You can rename a file and its extension at any time and not change the data.*

It is not obvious what is in the file. Unless you made the file, or someone has told you what is in it, the only thing you can do is inspect the file manually. Let's look at the number of lines and the first 10 lines here.

A conventional way to do this is in a shell with the `head` command. In Jupyter you can prefix a line with `!` to indicate it is a shell command





In [ ]:
! head raman.txt



Alternatively, you can use a magic cell. Here instead of the first few lines, we look at the last few lines with the `tail` command.



In [ ]:
%%bash
tail raman.txt



We can also do that in Python by opening the file, reading the lines and then closing the file. We print the number of lines and the first 10 here.



In [ ]:
f = open('raman.txt')
lines = f.readlines()
f.close()

print(f'{len(lines)} lines in the file')
print(lines[0:10])
print(''.join(lines[0:10]))



You might notice the "\t" and "\n" characters. The first is a tab, and the second is a new line (the new line is what defines a line, each line ends with this character). This looks like a *tab-delimited* text file, with two columns where the numbers are separated by tabs. Unfortunately, there is no information about *what* the columns are. You have to know from some other context. This file is from a Raman spectrometer, and the first column is the wavenumber, and the second column is the intensity.

It is important that you remember to close the file. If you are reading lots of files, you can run into operating system errors if you open too many open files. Python is at your service with a *context manager*. This approach will *automatically* close the file when it goes of scope.





In [ ]:
with open('raman.txt') as f:
    # f is only open and available inside this body
    lines = f.readlines()
    print(f'Inside: {f.closed}')

print(f'Outside: {f.closed}')



### Simple loading into numpy





There is an easy way for us to read this file so we can get the data into Python for analysis, via `np.loadtxt`. Let's start with the simplest method. We simply use the filename as the only argument.





In [ ]:
import numpy as np
data = np.loadtxt('raman.txt')
data



We can see the shape here, which confirms that we have read the number of lines, and that there are two columns.





In [ ]:
data.shape



It is often helpful to separate this array into variables. The first column is the wavenumber and the second column is the intensity. We use indexing to extract these into variables. This is a 2D array, and we index using a notation of `[row, col]` where `row` indicates which row(s) we want, and `col` indicates which column(s) we want. If you use a `:` it means all of them. Indexing can be very complex ([https://docs.scipy.org/doc/numpy/reference/arrays.indexing.html](https://docs.scipy.org/doc/numpy/reference/arrays.indexing.html)), and we will return to it many times as we do more and more sophisticated things.





In [ ]:
wavenumber = data[:, 0]
intensity = data[:, 1]

(wavenumber.shape, intensity.shape)



Now, these variables are 1D arrays.

Alternatively, you can *unpack* an array. When you iterate over an array, you iterate over its rows. If we transpose this data, we will change it from two columns with a lot of rows into two rows with a lot of columns.





In [ ]:
wavenumber, intensity = data.T
(wavenumber.shape, intensity.shape)



Here are some examples of indexing these arrays:





In [ ]:
# first and last element
(wavenumber[0], wavenumber[-1])



In [ ]:
# first five and last 5 elements
(intensity[0:5], intensity[-5:])



You might be tempted to get the last five elements like this, but it is wrong. Remember in indexing the last element in the slice is not included. Since you specify -1 in the slice it means up to *but not including* the last element. In contrast, `[-5:]` means from the fifth to last element to the end, *including the end*.





In [ ]:
intensity[-5:-1]



There are even fancier ways to slice this, e.g. to get every other point or every nth point, or even a set of selected points. We will see these later.

That is a lot of data. How do we see what is in it? Visualization.





### Simple visualization of the data





[matplotlib](https://matplotlib.org/) is the primary plotting tool we will use. There are a few others that are popular, especially [seaborn](https://seaborn.pydata.org/). It is good advice that when you want to make a plot, a good way to do it is find an example plot like it and see if you can adapt that code to your data.

-   **matplotlib sample plots:** [https://matplotlib.org/tutorials/introductory/sample_plots.html](https://matplotlib.org/tutorials/introductory/sample_plots.html)
-   **matplotlib gallery:** [https://matplotlib.org/gallery/index.html](https://matplotlib.org/gallery/index.html)
-   **seaborn gallery:** [https://seaborn.pydata.org/examples/index.html](https://seaborn.pydata.org/examples/index.html)

There are many other plotting libraries, e.g. [plotly](https://plot.ly/) and [altair](https://altair-viz.github.io/gallery/index.html#gallery-category-interactive-charts) that are more suitable for web graphics. These have much more complex APIs and it takes a long time to get familiar with them.

We will start simple. Let's start by telling the notebook how to use matplotlib and then load it. The line starting with a % is called a [magic command](https://ipython.readthedocs.io/en/stable/interactive/magics.html), and it is only valid in Jupyter-Python, or a jupyter notebook. It is *not* valid Python. It tells the Jupyter kernel how to handle plots, and in this case we are telling it to put the plot inline as a PNG image in the notebook.





In [ ]:
import matplotlib.pyplot as plt



We can make a plot as simple as this:





In [ ]:
plt.plot(wavenumber, intensity)
plt.xlabel('Wavenumber')
plt.ylabel('Intensity');



Now, we have a graphical representation of this data set. You can interact with it in the notebook. You can see the coordinates, and zoom in on regions. Suppose we want to make sure we *reproducibly* zoom in to a region though. That requires us to script it so that you have a written record of what you did.

Let's make a plot of a subset of the spectrum, say the points where the wavenumber is at least 1000, and less than 1500. To do that, we need a way to select the data subset. We can use indexing for this purpose. We define a *boolean* array where it is True for the points we want, and False for the ones we do not want. The comparison operators will do this, and the logical & operator combines the two conditions. Note these are all elementwise operations.





In [ ]:
# indices of points where 1000 >= wavenumber > 1500
# True means it meets the criteria, False means it does not
ind = (wavenumber >= 1000) & (wavenumber < 1500)
ind



These two arrays have the same shape, that is we evaluated the condition on every single point.





In [ ]:
ind.shape, wavenumber.shape



How many points did we get? We can find out by counting the number of True values. In Python, True=1, and False=0.





In [ ]:
True + True



So, we get the number of True values just by summing them here.





In [ ]:
np.sum(ind)



We can also plot the values like this:





In [ ]:
plt.plot(wavenumber, ind)
plt.xlabel('wavenumber')
plt.ylabel('ind');



These are all ways to check that we have the indices we want. Now we use that array to select the *portion* of the data we want. When we use the Boolean array to index with, the result is a shorter array containing only values where the Boolean array was True.





In [ ]:
wavenumber[ind].shape



We have to index *both* the x and y data to make this plot, because we can only plot arrays with the matching dimensions.





In [ ]:
plt.figure()
plt.plot(wavenumber[ind], intensity[ind])
plt.xlabel('Wavenumber')
plt.ylabel('Intensity');



### Summary





There was a lot of explanation above, so the brevity of this code may have been lost. Here is what we need to read the data and plot it (assuming you have already done the imports).





In [ ]:
data = np.loadtxt('raman.txt')

wavenumber, intensity = data.T  # the transpose has data in rows for unpacking
ind = (wavenumber >= 1000) & (wavenumber < 1500)

plt.figure()
plt.plot(wavenumber[ind], intensity[ind])
plt.xlabel('Wavenumber')
plt.ylabel('Intensity');



In this simple example we have learned how to:

1.  Retrieve data from a url into a local file
2.  Read a local file of tab-delimited data into numpy arrays
3.  Visualize the arrays
4.  Select a subset of the arrays and visualize the subset.





## Data formats





The data in the previous example was loaded from a tab-delimited text file. There are many other kinds of files you might have to read from.





### Simple delimited text files





In a delimited text file, the data is represented in each line, with the values separated by a *delimiter*. In the tab-delimited data, tab characters are the separators. The other common delimiter is a comma, and these files are often called comma-separated value (CSV) files. For now, we limit the discussion to files containing numbers that are separated by commas.





In [ ]:
with open('raman.csv') as f:
    lines = f.readlines()

print(len(lines))
print(lines[0:10])



You can see the numbers in each line are now separated by commas. `np.loadtxt` works for this file too, but we have to specify that the delimiter is a comma. See the [documentation](https://docs.scipy.org/doc/numpy/reference/generated/numpy.loadtxt.html) on `np.loadtxt` for more options.

Note: CSV is surprisingly complex, and there is a [dedicated library](https://docs.python.org/3/library/csv.html) for parsing it. We will stick to simple numerical csv files, which are easy to load in numpy.





In [ ]:
np.loadtxt('raman.csv', delimiter=',')



Let's do one more thing that is convenient here, which is we directly *unpack* the columns into variable names:





In [ ]:
wave, intensity = np.loadtxt('raman.csv', delimiter=',', unpack=True)
plt.plot(wave, intensity)
plt.xlabel('Wavenumber')
plt.ylabel('intensity');



### more complex delimited files





This data is located at [https://www.itl.nist.gov/div898/handbook/datasets/MODEL-4_4_4.DAT](https://www.itl.nist.gov/div898/handbook/datasets/MODEL-4_4_4.DAT). There is a copy of the file in this directory.

If we examine the first few lines of this file, it appears that the first two lines are considered headers that tell you what is in the file. We need to skip these when reading the data. Also, each line appears as a string, with the values lining up in columns. This is sometimes called a fixed delimiter file.





In [ ]:
with open('p-t.dat') as f:
    print(''.join(f.readlines()[0:5]))



I did something kind of fancy there. First, we read the lines and slice out the first 5 of them. That gives us a list of 5 lines. Then, I rejoin them so we get a single string, and then I printed it. That is to avoid getting two newlines (one from the line, and one from the print). We can tell print not to add a new line like this.





In [ ]:
with open('p-t.dat') as f:
    for line in f.readlines()[0:5]:
        print(line, end='')



Either way, you can see there are two non-data lines here. We can still load this file with `np.loadtxt` we just tell it to skip the first two lines.





In [ ]:
d = np.loadtxt('p-t.dat', skiprows=2)
d



You may note, however, that the first two columns are floats, and not integers. We can fix that (if it matters), at the cost of some complexity. We specify the format of each column in the `dtype` argument. This leads to a structured data array ([https://docs.scipy.org/doc/numpy/user/basics.rec.html](https://docs.scipy.org/doc/numpy/user/basics.rec.html)).

The dtype formats are documented at [https://docs.scipy.org/doc/numpy/reference/arrays.dtypes.html#arrays-dtypes-constructing](https://docs.scipy.org/doc/numpy/reference/arrays.dtypes.html#arrays-dtypes-constructing). The ones we use here are for 32-bit integers and floats.





In [ ]:
d = np.loadtxt('p-t.dat', skiprows=2,
               dtype={'names': ('run order', 'day', 'ambient temperature', 'temperature', 'pressure', 'fitted value', 'residual'),
                      'formats': ('i4', 'i4', 'f4', 'f4', 'f4', 'f4', 'f4')})
d



This array should look different to you. First, each row is in (), and the dtype at the end looks different, with names and formats.

One benefit of this is you can now use human readable names to select the various columns. We will return to this idea later when we learn about Pandas.





In [ ]:
plt.plot(d['temperature'], d['pressure'], 'b. ')
plt.xlabel('Temperatue')
plt.ylabel('pressure');



Note that this is not a 2D array, even though it looks like one! It is considered a 1D array of records.





In [ ]:
d.shape



It is not possible to slice this array like we did before. Instead we select columns by name.





In [ ]:
d['temperature']



#### Summary





This data file was a text file, with a *header* that described some information about the data. The data itself was still delimited, but each column had a different type of data, e.g. in this case integers and floats. Some files may also have string data in columns. As the data gets more heterogeneous, it gets more challenging to read it into variables. In the worst case scenario, you can write a custom data parser, but this is hard work that should usually be avoided if possible.





### json data





json (javascript object notation) is another convenient data format ([https://www.json.org/json-en.html](https://www.json.org/json-en.html)). This data format stores data in a "key=value" format, and when you load it, you get a dictionary of the data. Let's see the first few characters of this file. Why the first few characters? A json file may be only one line as defined by newlines.





In [ ]:
with open('raman.json') as f:
    print(len(f.readlines()))



But you can see there are a lot of characters:





In [ ]:
with open('raman.json') as f:
    print(len(f.read()))



Let's see the first 50 of the characters.





In [ ]:
with open('raman.json') as f:
    print(''.join(f.read()[0:50]))



Here, "wavenumber" is a key, and it has a value (the thing on the other side of the :) that is an array of numbers. Luckily, it is easy to load this in Python with the json library.





In [ ]:
import json

with open('01-raman.json') as f:
    d = json.load(f)

type(d), d.keys()



Note that this does not read the data into numpy arrays. We have a list for these instead. We can convert them to arrays if we need to.





In [ ]:
type(['wavenumber'])



In [ ]:
plt.plot(d['wavenumber'], d['intensity'])
plt.xlabel('Wavenumber')
plt.ylabel('Intensity');



It is fine to plot with these lists, but if you need to do any array operations or analysis on them, you may need to convert them to arrays first (although many numpy functions work on lists just fine).





### There are many more data formats





There are many more formats. For most of these, there are standard libraries for reading them data into an array form that you can use. You should usually search for a library that can read your data first, rather than try to implement your own parser.

-   **netcdf:** [https://unidata.github.io/netcdf4-python/netCDF4/index.html](https://unidata.github.io/netcdf4-python/netCDF4/index.html)
-   **hdf5:** [https://www.h5py.org/](https://www.h5py.org/)
-   **xml:** [https://docs.python.org/3/library/xml.html](https://docs.python.org/3/library/xml.html)
-   **matlab .mat files:** [https://docs.scipy.org/doc/scipy/reference/generated/scipy.io.loadmat.html](https://docs.scipy.org/doc/scipy/reference/generated/scipy.io.loadmat.html)
-   **binary data files:** [https://www.devdungeon.com/content/working-binary-data-python](https://www.devdungeon.com/content/working-binary-data-python)
-   **scipy io routines:** [https://docs.scipy.org/doc/scipy/reference/io.html](https://docs.scipy.org/doc/scipy/reference/io.html)
-   **molecular simulation files:** [https://wiki.fysik.dtu.dk/ase/ase/io/io.html#module-ase.io](https://wiki.fysik.dtu.dk/ase/ase/io/io.html#module-ase.io)
-   **yaml:** [https://pyyaml.org/wiki/PyYAMLDocumentation](https://pyyaml.org/wiki/PyYAMLDocumentation)
-   **config files:** [https://docs.python.org/3/library/configparser.html](https://docs.python.org/3/library/configparser.html)
-   **Excel:** [https://openpyxl.readthedocs.io/en/stable/index.html](https://openpyxl.readthedocs.io/en/stable/index.html)





## Summary





Today the focus was on orienting ourselves to the main ideas of reading data files. These are:

1.  What kind of data file is it?
2.  Is there a library function that makes it easy to read?
3.  How do you get the data into arrays and variables for further analysis.

We also examined how to make simple visualizations of the data.



